# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3486, 3487, 3488]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3486,
    "name": "Pygray",
    "account": {
      "id": 3704,
      "number": "14254-7",
      "agency": "0001-1",
      "balance": 3000.0,
      "limit": 10000.0
    },
    "card": {
      "id": 3382,
      "number": "xxxx xxxx xxxx 2024",
      "limit": 5000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3487,
    "name": "Pylenoy",
    "account": {
      "id": 3705,
      "number": "14255-8",
      "agency": "0001-1",
      "balance": 10000.0,
      "limit": 100000.0
    },
    "card": {
      "id": 3383,
      "number": "xxxx xxxx xxxx 2025",
      "limit": 20000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3488,
    "name": "Pymilo",
    "account": {
      "id": 3706,
      "number": "14256-9",
      "agency": "0001-1",
      "balance": 1.0,
      "limit": 500.0
    },
    "card": {
      "id": 3384,
      "number": "xxxx xxxx xxxx 2026",
      "limit": 200.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Utilize a API do OpenAI GPT-3.5-turbo para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
!pip install openai

In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO'

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em marketing bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos, \
          levando em concideração o limite do cartão de crédito de {user['card']['limit']} e salda \
          na conta bancária de {user['account']['balance']}, sem citar valores (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Invista e construa um futuro financeiro sólido!
Invista hoje e garanta um futuro financeiramente seguro!
Invista com inteligência para garantir um futuro financeiro estável.


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Pygray updated? True!
User Pylenoy updated? True!
User Pymilo updated? True!
